In [29]:
import polars as pl
from glob import glob   

In [30]:
#all unnested data except data1.csv
folder_path = 'data/unnested_data'
files = [f"{folder_path}/data{i}.csv" for i in range(2,17)]
#read data1.csv
df_res = pl.read_csv(f"{folder_path}/data1.csv",parse_dates=True)
for file in files:
    df = pl.read_csv(file,parse_dates=True)
    df_res = pl.concat([df_res,df])
df_crapauduc = df_res

In [31]:
import json
file_path = "data/meteo/airport_LSGG_2017-02-01-1516"

with open(file_path) as f:
    data = json.load(f)
    print(json.dumps(data, indent=4))

{
    "response": {
        "version": "0.1",
        "termsofService": "http://www.wunderground.com/weather/api/d/terms.html",
        "features": {
            "yesterday": 1
        }
    },
    "history": {
        "date": {
            "pretty": "January 31, 2017",
            "year": "2017",
            "mon": "01",
            "mday": "31",
            "hour": "00",
            "min": "00",
            "tzname": "Europe/Zurich"
        },
        "utcdate": {
            "pretty": "January 30, 2017",
            "year": "2017",
            "mon": "01",
            "mday": "30",
            "hour": "23",
            "min": "00",
            "tzname": "UTC"
        },
        "observations": [
            {
                "date": {
                    "pretty": "12:00 AM CET on January 31, 2017",
                    "year": "2017",
                    "mon": "01",
                    "mday": "31",
                    "hour": "00",
                    "min": "00",
                

There isn't any documentation, i reversed engineered the data meanings, here is a snippet of the data, the explanations follows.

    {     
        "fog": "1",    
        "rain": "1",    
        "snow": "0",    
        "snowfallm": "",    
        "snowfalli": "",    
        "monthtodatesnowfallm": "",    
        "monthtodatesnowfalli": "",    
        "since1julsnowfallm": "",    
        "since1julsnowfalli": "",    
        "snowdepthm": "",    
        "snowdepthi": "",    
        "hail": "0",    
        "thunder": "0",    
        "tornado": "0",    
        "meantempm": "5",    
        "meantempi": "42",    
        "meandewptm": "5",    
        "meandewpti": "42",    
        "meanpressurem": "1018.17",    
        "meanpressurei": "30.07",    
        "meanwindspdm": "3",    
        "meanwindspdi": "2",    
        "meanwdire": "WSW",    
        "meanwdird": "253",    
        "meanvism": "8.2",    
        "meanvisi": "5.1",    
        "humidity": "97",    
        "maxtempm": "7",    
        "maxtempi": "46",    
        "mintempm": "3",    
        "mintempi": "37",    
        "maxhumidity": "100",    
        "minhumidity": "81",    
        "maxdewptm": "7",    
        "maxdewpti": "45",    
        "mindewptm": "4",    
        "mindewpti": "38",    
        "maxpressurem": "1021",    
        "maxpressurei": "30.14",    
        "minpressurem": "1017",    
        "minpressurei": "30.04",    
        "maxwspdm": "11",    
        "maxwspdi": "7",    
        "minwspdm": "0",    
        "minwspdi": "0",    
        "maxvism": "20.0",    
        "maxvisi": "12.0",    
        "minvism": "0.9",    
        "minvisi": "0.6",    
        "gdegreedays": "0",    
        "heatingdegreedays": "24",    
        "coolingdegreedays": "0",    
        "precipm": "4.0",    
        "precipi": "0.16"
    }

## What do we have ? 
There is a measure, for example precipm and then there is the same measure with a i after : precipi for example.
The first one is in metric and the second one is in imperial units. (ffs, throw these garbage units away)  
For precipitation, we have mm and inches. Another example ? meanpressurem and meanpressurei. The first is in millibar like everyone would expect and the second one is in inches of mercury (wtf).

There is some empty files too, for example airport_LSGG_2017-04-07-1517 who contains an api error. renamed to buggy to avoid confusion.



In [32]:
import json
from datetime import datetime 
from typing import Iterable

def create_df_from_json(paths :Iterable[str]):
    dates = []
    temperatures = []
    speeds = []
    pressures = []
    precipitations = []
    rains = []
    humiditys = []
    for json_file_path in paths:
        with open(json_file_path) as meteo_file:
            try:
                meteo_data = json.load(meteo_file)
                dsmry = meteo_data['history']['dailysummary'][0] #who tf puts a dict in a 1 element list ??
                full_date = dsmry['date']
                y,m,d = full_date['year'], full_date['mon'], full_date['mday']   
                mean_temperature =  dsmry['meantempm']
                mean_wind_speed = dsmry['meanwindspdm']
                mean_pressure = dsmry['meanpressurem']
                precipitation_mm =  dsmry['precipm']
                has_rain = "1" in dsmry['rain']  #needs to be a string for some reason
                humidity =  dsmry['humidity']
                dates.append(f"{y}-{m}-{d}")
                temperatures.append(int(mean_temperature))
                speeds.append(int(mean_wind_speed))
                pressures.append(float(mean_pressure))
                precipitations.append(float(precipitation_mm))
                humiditys.append(int(humidity))
                rains.append(has_rain)
            except :
                print(f"Error with {json_file_path}")

    data = [pl.Series('date',dates), pl.Series('temperature_celcius',temperatures), pl.Series('wind_km/h',speeds), pl.Series('pressure_millibar',pressures), pl.Series('precipitation_mm',precipitations), pl.Series('rain',rains), pl.Series('humidity',humiditys)]
    df = pl.DataFrame(data)
    return df.with_column(pl.col('date').str.strptime(pl.Date,fmt='%Y-%m-%d'))


In [33]:
files = glob('data/meteo/airport_*')
df_meteo = create_df_from_json(files)
#df_meteo.to_csv('data/meteo.csv')
df_meteo.filter(pl.col('rain') == True)

date,temperature_celcius,wind_km/h,pressure_millibar,precipitation_mm,rain,humidity
date,i64,i64,f64,f64,bool,i64
2017-02-24,6,13,1014.5,0.0,true,53
2017-03-07,4,7,1018.82,2.0,true,84
2017-04-15,12,6,1018.21,0.0,true,58
2017-04-30,9,4,1010.28,0.1,true,61
2017-02-04,6,12,1006.39,6.0,true,75
2017-06-06,14,9,1011.77,11.0,true,79
2017-06-26,20,4,1013.85,1.0,true,65
2017-05-20,11,5,1022.59,0.2,true,76
2017-02-08,4,6,1018.04,1.0,true,94
